<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/NeuralBeagle_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 27.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
import transformers
import torch

In [3]:
function = """
def checkGPU(tensorflow):
    if tensorflow == True:
        import tensorflow as tf
        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices('GPU')))
    else:
        import torch
        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())
"""

In [4]:
model = "mlabonne/NeuralBeagle14-7B"

In [5]:
messages = [
    {
        "role": "system",
        "content": "You are an informative programming assistant that understands the working of a function and replies with it's documentation in markdown syntax."
    },
    {
        "role": "user",
        "content": f"{function}"
    }
]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16, # Remember that float32 - CPU and float16 - GPU, keep this in mind if RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [7]:
%%time
outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>system
You are an informative programming assistant that understands the working of a function and replies with it's documentation in markdown syntax.</s>
<s>user

def checkGPU(tensorflow):
    if tensorflow == True:
        import tensorflow as tf
        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices('GPU')))
    else:
        import torch
        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())
</s>
<s>assistant
 Here is the documentation for the `checkGPU` function in Markdown format:

```
# checkGPU(tensorflow: bool)

A function to determine the availability of a GPU depending on the chosen framework (either TensorFlow or PyTorch).

## Parameters
- `tensorflow`: A boolean value indicating whether the user wants to check for GPU availability with TensorFlow (`True`) or PyTorch (`False`).

## Returns
- If `tensorflow == True`:
  - Prints the number of GPUs available with TensorFlow using the `tf.config.list